In [ ]:
%pip install tensorflow tensorflow-gpu opencv-python matplotlib

In [ ]:
!pip list

In [ ]:
import tensorflow as tf
import os
import cv2 
import imghdr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
data_dir='./dataset/train/'

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

In [ ]:
os.listdir(data_dir)
os.listdir(os.path.join(data_dir,'CNV'))

In [ ]:
# i=0
# for imageClass in os.listdir(data_dir):
#     print(imageClass)
#     file = open("./dataDimentions"+""+"_"+imageClass+".csv","w")
#     file.write("length,breadth,depth\n")
#     for image in os.listdir(os.path.join(data_dir,imageClass)):
#         pathToImage=os.path.join(data_dir,imageClass,image)
#         # print(pathToImage)
#         try:
#             img = cv2.imread(pathToImage)
#             s = str(img.shape[0])+","+str(img.shape[1])+","+str(img.shape[2])+"\n"
#             file.write(s)             
#             tip = imghdr.what(pathToImage)
#             if(tip!="jpeg"):
#                 print(pathToImage)
#             if(imageClass=="DME"):
#                 print(i,"==>",s)
#                 i+=1
#         except:
#             print("image does not exist")
        

In [ ]:
# tf.data.Dataset??
# tf.keras.utils.image_dataset_from_directory??

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(data_dir)
data_iterator = data.as_numpy_iterator()
data_iterator
# batch = data_iterator.next()

In [ ]:
len(batch)
batch[0].shape
# batch[1]

In [ ]:
fig,ax = plt.subplots(ncols=8,figsize=(20,20))
for idx,img in enumerate(batch[0][:8]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
scaled = batch[0]/255
scaled.max()

In [ ]:
data = data.map(lambda x,y:(x/255,y))
scaled_iterator = data.as_numpy_iterator().next()

In [ ]:
scaled_iterator[0].min()

In [ ]:
cnvData = pd.read_csv("./dataDimentions_CNV.csv")
# plt.hist(cnvData)
cnvData["total"] = cnvData["breadth"]+ cnvData["length"] + cnvData["depth"]
# plt.hist(cnvData["total"])

In [ ]:
trainSize = int(len(data)*.07)
valSize = int(len(data)*.02)
testSize = int(len(data)*.01)+1

In [ ]:

len(data) == trainSize + valSize + testSize
valSize

In [ ]:
trainData = data.take(trainSize)
valData = data.skip(trainSize).take(valSize)
testData = data.skip(trainSize+valSize).take(testSize)

In [ ]:
len(trainData)
len(testData)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout

In [ ]:
model = Sequential()
model1 = Sequential()

In [ ]:
model.add(Conv2D(16,(3,3),1,activation='relu',input_shape=(256,256,3)))
model.add(MaxPool2D())

model.add(Conv2D(32,(3,3),1,activation='relu'))
model.add(MaxPool2D())

model.add(Conv2D(16,(3,3),1,activation='relu'))
model.add(MaxPool2D())

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dense(4,activation='sigmoid'))

model1.add(model)
model1.add(Conv2D(32,(3,3),1,activation='relu',input_shape=(256,256,3)))
model1.add(MaxPool2D())

model1.add(Conv2D(64,(3,3),1,activation='relu'))
model1.add(MaxPool2D())

model1.add(Conv2D(16,(3,3),1,activation='relu'))
model1.add(MaxPool2D())

model1.add(Flatten())
model1.add(Dense(64,activation='relu'))
model1.add(Dense(4,activation='softmax'))

In [ ]:
model1.compile(optimizer='SGD',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
logDir = 'logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logDir)
hist = model.fit(trainData,epochs=5,validation_data=valData,callbacks=[tensorboard_callback])

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'],color='teal',label='loss')
plt.plot(hist.history['val_loss'],color='orange',label='val_loss')
fig.suptitle('loss',fontsize=20)
plt.legend(loc='upper left')
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'],color='teal',label='acc')
plt.plot(hist.history['val_accuracy'],color='orange',label='val_acc')
fig.suptitle('Accuracy',fontsize=20)
plt.legend(loc='upper left')
plt.show()

## Evaluation